# 8. SQL Stored Procedures: Full Concept with Example

A stored procedure is a set of SQL statements that can be executed on the database server. They are used to encapsulate and reuse SQL code, improve performance, and ensure data integrity and security.

### Key Concepts

1. **Creation**: Define a stored procedure using the `CREATE PROCEDURE` statement.
2. **Execution**: Run a stored procedure using the `EXEC` or `CALL` statement.
3. **Parameters**: Pass values to the procedure using input (`IN`), output (`OUT`), or input-output (`INOUT`) parameters.
4. **Logic**: Include control-of-flow constructs (e.g., `IF`, `WHILE`, `LOOP`) within the procedure.
5. **Transactions**: Handle transactions within the procedure using `BEGIN`, `COMMIT`, and `ROLLBACK`.

### Example

Let's create a basic example to illustrate stored procedures in SQL.

### 1. Setup: Creating a Sample Table

We'll start by creating a table named `Employees`.

```sql
CREATE TABLE Employees (
    EmployeeID INT PRIMARY KEY,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    Salary DECIMAL(10, 2),
    DepartmentID INT
);
```

### 2. Inserting Sample Data

Insert some sample data into the `Employees` table.

```sql
INSERT INTO Employees (EmployeeID, FirstName, LastName, Salary, DepartmentID)
VALUES
(1, 'John', 'Doe', 60000, 101),
(2, 'Jane', 'Smith', 75000, 102),
(3, 'Jim', 'Brown', 50000, 103);
```

### 3. Creating a Stored Procedure

Create a stored procedure to get the details of an employee by their `EmployeeID`.

```sql
CREATE PROCEDURE GetEmployeeDetails(IN empID INT)
BEGIN
    SELECT EmployeeID, FirstName, LastName, Salary, DepartmentID
    FROM Employees
    WHERE EmployeeID = empID;
END;
```

### 4. Executing the Stored Procedure

Execute the `GetEmployeeDetails` procedure to retrieve details for an employee with `EmployeeID` 1.

```sql
CALL GetEmployeeDetails(1);
```

### 5. Creating a Procedure with Output Parameters

Create a procedure to calculate the total salary of all employees in a specific department.

```sql
CREATE PROCEDURE GetTotalSalaryByDepartment(IN deptID INT, OUT totalSalary DECIMAL(10, 2))
BEGIN
    SELECT SUM(Salary) INTO totalSalary
    FROM Employees
    WHERE DepartmentID = deptID;
END;
```

### 6. Executing the Procedure with Output Parameters

Declare a variable to hold the output and call the procedure.

```sql
SET @totalSalary DECIMAL(10, 2);

CALL GetTotalSalaryByDepartment(101, @totalSalary);

SELECT @totalSalary AS TotalSalary;
```

### 7. Using Control-of-Flow Statements

Create a procedure to give a raise to employees based on their performance rating.

```sql
CREATE PROCEDURE GiveRaise(IN empID INT, IN rating CHAR(1))
BEGIN
    IF rating = 'A' THEN
        UPDATE Employees
        SET Salary = Salary * 1.10
        WHERE EmployeeID = empID;
    ELSEIF rating = 'B' THEN
        UPDATE Employees
        SET Salary = Salary * 1.05
        WHERE EmployeeID = empID;
    ELSE
        UPDATE Employees
        SET Salary = Salary * 1.02
        WHERE EmployeeID = empID;
    END IF;
END;
```

### 8. Executing the Control-of-Flow Procedure

Give a raise to an employee with `EmployeeID` 2 based on a performance rating of 'A'.

```sql
CALL GiveRaise(2, 'A');
```

### 9. Handling Transactions

Create a procedure to transfer an employee to a new department and ensure all operations are consistent using transactions.

```sql
CREATE PROCEDURE TransferEmployee(IN empID INT, IN newDeptID INT)
BEGIN
    DECLARE EXIT HANDLER FOR SQLEXCEPTION
    BEGIN
        ROLLBACK;
    END;

    START TRANSACTION;

    UPDATE Employees
    SET DepartmentID = newDeptID
    WHERE EmployeeID = empID;

    COMMIT;
END;
```

### 10. Executing the Transaction Procedure

Transfer an employee with `EmployeeID` 3 to a new department `104`.

```sql
CALL TransferEmployee(3, 104);
```


### 11. Error Handling in SQL Stored Procedures

Error handling in SQL stored procedures ensures that your procedures can gracefully handle and respond to errors. Commonly used constructs for error handling include `DECLARE HANDLER` for handling exceptions and `SIGNAL` for raising errors. Here, we'll discuss how to use these constructs in the context of our `Employees` table.

### 1. Setup: Creating the Employees Table and Inserting Data

If you haven't created the `Employees` table yet, here's the SQL code to do so:

```sql
CREATE TABLE Employees (
    EmployeeID INT PRIMARY KEY,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    Salary DECIMAL(10, 2),
    DepartmentID INT
);

INSERT INTO Employees (EmployeeID, FirstName, LastName, Salary, DepartmentID)
VALUES
(1, 'John', 'Doe', 60000, 101),
(2, 'Jane', 'Smith', 75000, 102),
(3, 'Jim', 'Brown', 50000, 103);
```

### 2. Error Handling with DECLARE HANDLER

Let's create a stored procedure to update an employee's salary. We'll include error handling to roll back the transaction if an error occurs.

```sql
CREATE PROCEDURE UpdateEmployeeSalary(
    IN empID INT, 
    IN newSalary DECIMAL(10, 2)
)
BEGIN
    DECLARE EXIT HANDLER FOR SQLEXCEPTION
    BEGIN
        ROLLBACK;
        SELECT 'An error occurred while updating the salary.' AS ErrorMessage;
    END;

    START TRANSACTION;
    
    UPDATE Employees
    SET Salary = newSalary
    WHERE EmployeeID = empID;
    
    COMMIT;
END;
```

### 3. Error Handling with DECLARE CONTINUE HANDLER

Let's create a procedure to increase the salaries of all employees. We'll include error handling to continue processing even if an error occurs for a specific employee.

```sql
CREATE PROCEDURE IncreaseAllSalaries(
    IN percentageIncrease DECIMAL(5, 2)
)
BEGIN
    DECLARE CONTINUE HANDLER FOR SQLEXCEPTION
    BEGIN
        -- Log the error or take other actions
        SELECT 'An error occurred while updating the salary for an employee.' AS ErrorMessage;
    END;

    DECLARE empID INT DEFAULT 1;
    DECLARE maxEmpID INT;

    -- Find the maximum EmployeeID
    SELECT MAX(EmployeeID) INTO maxEmpID FROM Employees;

    raise_loop: LOOP
        IF empID > maxEmpID THEN
            LEAVE raise_loop;
        END IF;

        -- Update salary
        UPDATE Employees
        SET Salary = Salary * (1 + percentageIncrease / 100)
        WHERE EmployeeID = empID;

        -- Move to the next employee
        SET empID = empID + 1;
    END LOOP raise_loop;
END;
```

### 4. Raising Custom Errors with SIGNAL

Let's create a procedure that raises a custom error if an employee's salary is set to a value below a certain threshold.

```sql
CREATE PROCEDURE SetEmployeeSalary(
    IN empID INT, 
    IN newSalary DECIMAL(10, 2)
)
BEGIN
    IF newSalary < 30000 THEN
        SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = 'Salary must be at least 30000.';
    ELSE
        UPDATE Employees
        SET Salary = newSalary
        WHERE EmployeeID = empID;
    END IF;
END;
```

### 5. Using EXIT HANDLER for Cleanup

Let's create a procedure to transfer an employee to a new department and handle errors by rolling back the transaction and performing cleanup actions.

```sql
CREATE PROCEDURE TransferEmployee(
    IN empID INT, 
    IN newDeptID INT
)
BEGIN
    DECLARE EXIT HANDLER FOR SQLEXCEPTION
    BEGIN
        ROLLBACK;
        -- Additional cleanup actions can be performed here
        SELECT 'An error occurred during the transfer. The transaction has been rolled back.' AS ErrorMessage;
    END;

    START TRANSACTION;
    
    UPDATE Employees
    SET DepartmentID = newDeptID
    WHERE EmployeeID = empID;
    
    COMMIT;
END;
```

### 6. Executing the Procedures

You can execute any of the above procedures to see error handling in action. For example:

```sql
CALL UpdateEmployeeSalary(1, 65000);
CALL IncreaseAllSalaries(5);
CALL SetEmployeeSalary(2, 25000); -- This will raise a custom error
CALL TransferEmployee(3, 104);
```

### Conclusion

Error handling in SQL stored procedures is crucial for robust database operations. By using constructs like `DECLARE HANDLER` and `SIGNAL`, you can ensure that your procedures handle exceptions gracefully and maintain data integrity.

### IN, OUT, INOUT Parameters in SQL Stored Procedures

SQL stored procedures can use parameters to pass values into the procedure, return values from the procedure, or both. The types of parameters are:

1. **IN**: Passes a value into the procedure. The value of an `IN` parameter can be changed within the procedure, but the change is not reflected back to the caller.
2. **OUT**: Returns a value from the procedure. The caller can retrieve the value of an `OUT` parameter after the procedure completes.
3. **INOUT**: Passes a value into the procedure and returns a value back to the caller. The `INOUT` parameter can be used both for input and output.

### When to Use IN, OUT, and INOUT Parameters

- **IN** parameters are used when you need to provide input data to the procedure.
- **OUT** parameters are used when you need to return a value from the procedure to the caller.
- **INOUT** parameters are used when you need to provide input data to the procedure and also retrieve output data from the procedure.


- **IN Parameters**: Used for input values.
- **OUT Parameters**: Used for output values.
- **INOUT Parameters**: Used for both input and output values.


#### Prepared By,
Ahamed Basith